In [1]:
from __future__ import print_function
import sys
import numpy as np
import tables as h5
#import pandas as pd
#pd.options.display.max_rows = 30
import matplotlib.pyplot as plt
%matplotlib notebook
import gvar as gv
import fkfpi_lib as flib
import input_params as ip
print("python version:", sys.version)
#print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("gvar   version:", gv.__version__)

python version: 3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy  version: 1.16.1
gvar   version: 9.0.2


In [2]:
switches = ip.switches
# To override default choices of fit type and ensembles, set override=True and adjust here
override=False
if override:
    switches['ensemble'] = [
        'a15m400','a12m400','a09m400',
        'a15m350','a12m350','a09m350',
        'a15m310','a12m310','a09m310',
        'a15m220','a12m220','a09m220','a12m220S','a12m220L',
        'a12m130','a15m130']
    switches['ansatz'] = dict()
    switches['ansatz']['FV'] = True # True turns on NLO FV correction
    switches['ansatz']['type'] = 'xpt' # Type of fit: 'xpt' or 'MA'
    switches['ansatz']['a2dm'] = 'individual' # avg or individual
    switches['scale'] = 'PK' # PP, PK, KK, LamChi = 4 * pi * sqrt(FA * FB)
    switches['debug'] = True

In [ ]:
# Set priors for Xpt
priors = ip.priors
if override:
    priors['L5']  = gv.gvar(0.0, 1.0)
    priors['s4']  = gv.gvar(0.0, 1.0)
    priors['c4']  = gv.gvar(0.0, 1.0)
    priors['d4']  = gv.gvar(0.0, 1.0)
    priors['e4']  = gv.gvar(0.0, 1.0)
    priors['s6']  = gv.gvar(0.0, 1.0)
    priors['sc6'] = gv.gvar(0.0, 1.0)
    priors['sd6'] = gv.gvar(0.0, 1.0)
    priors['se6'] = gv.gvar(0.0, 1.0)
# Physical parameters from PDG
phys_p = ip.phys_p
# FLAG FK/Fpi
flag_FKFpi = ip.flag_FKFpi

In [ ]:
data = h5.open_file('FK_Fpi_data.h5','r')
fmt_data = flib.format_h5_data(switches,data,priors)
data.close()

In [ ]:
data = pd.read_csv('./github_fkfpi_v2.csv')
hisq_params = pd.read_csv('./hisq_params.csv')
mixed_params = pd.read_csv('./github_mixed_v1.csv')
data.groupby('ensemble').describe()

In [ ]:
fmt_data = flib.format_data(switches,data,mixed_params,hisq_params,priors)
#print(fmt_data)
print(fmt_data['y'])

In [ ]:
jr = ['ju','ru','js','rs']
mrks = ['s','o','d','H']
shift = [-0.003,-0.001,0.001,0.003]
fig = plt.figure('w0_DMix',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
colors ={'a15m400':'#ec5d57','a15m350':'#ec5d57','a15m310':'#ec5d57','a15m220':'#ec5d57','a15m130':'#ec5d57',
        'a12m400':'#70bf41','a12m350':'#70bf41','a12m310':'#70bf41','a12m220':'#70bf41','a12m130':'#70bf41','a12m220S':'#70bf41','a12m220L':'#70bf41',
        'a09m400':'#51a7f9','a09m350':'#51a7f9','a09m310':'#51a7f9','a09m220':'#51a7f9'}
for i,e in enumerate(switches['ensemble']):
    epi = fmt_data['p']['mpi'][i]/fmt_data['x']['Lchi'][i]
    for j,mix in enumerate(jr):
        wMix = fmt_data['p']['wMix_'+mix][i]
        ax.errorbar(epi.mean+shift[j],wMix.mean,yerr=wMix.sdev,color=colors[e],marker=mrks[j],elinewidth=1,capsize=3,mfc='none')

In [ ]:
fit = flib.fit_data(switches,fmt_data,phys_p)
print(fit.format(maxline=True))

In [ ]:
fig = plt.figure('FK/Fpi',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
colors ={'a15m400':'#ec5d57','a15m350':'#ec5d57','a15m310':'#ec5d57','a15m220':'#ec5d57','a15m130':'#ec5d57',
        'a12m400':'#70bf41','a12m350':'#70bf41','a12m310':'#70bf41','a12m220':'#70bf41','a12m130':'#70bf41','a12m220S':'#70bf41','a12m220L':'#70bf41',
        'a09m400':'#51a7f9','a09m350':'#51a7f9','a09m310':'#51a7f9','a09m220':'#51a7f9'}
for i,e in enumerate(switches['ensemble']):
    epi = fmt_data['p']['mpi'][i]/fmt_data['x']['Lchi'][i]
    print('%8s %s %s' %(e,str(fmt_data['y'][i]),str(fit.fcn(fit.x,fit.p)[i])))
    ax.errorbar(epi.mean,fmt_data['y'][i].mean,yerr=fmt_data['y'][i].sdev,color=colors[e],marker='o',elinewidth=1,capsize=3,mfc='none')
    ax.errorbar(epi.mean+.005,fit.fcn(fit.x,fit.p)[i].mean,yerr=fit.fcn(fit.x,fit.p)[i].sdev,color=colors[e],marker='s',elinewidth=1,capsize=3,mfc='none')
fkp = flib.fkfpi_phys(phys_p,fit)
ax.errorbar((phys_p['mpi']/(4.*np.pi*phys_p['Fpi'])).mean,fkp.mean,yerr=fkp.sdev,color='magenta',marker='*',elinewidth=1,capsize=3,mfc='none')
ax.errorbar((phys_p['mpi']/(4.*np.pi*phys_p['Fpi'])).mean,1.193,yerr=0.003,color='black',marker='*',elinewidth=1,capsize=3,mfc='none')
plt.savefig('FKFpi_vs_epi.pdf',transparent=True)